# Masters of Finance Hotels



### 1.1 Purpose: 

#### This Notebook is to analyze the hotels within Hong Kong's financial for comparison via FourSquare against those within financial center, New York City.

### 1.2 Reason: 

#### This will help us as a single factor in determining overall attractiveness in terms of outside individuals traveling to these locations.

In [1]:
import bs4 as bs
from bs4 import BeautifulSoup

import requests
import urllib.request

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from IPython.display import Image 
from IPython.core.display import HTML 

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

import os

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 2.1 Analysis of Hong Kong

### Here the libraries I will need in order to complete the analysis of the citystate of Hong Kong were imported, and necessary packages installed.

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/Districts_of_Hong_Kong').text

soup = BeautifulSoup(source,'lxml')

My_table = soup.find('table',{'class':'wikitable sortable'})

links = My_table.find_all('a')

table = soup.find('table', attrs={'class':'wikitable'})

print('done')

done


### The groundwork for web scraping the Wikipedia article on the districts within Hong Kong has been laid

In [3]:
table_data = table.tbody.find_all("tr")

l = []
for tr in table_data:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=["District", "Chinese", "Population", 
                              "Area", "Density", "Region"])

print('done')

done


### The web scraping has been completed, and a data frame based on the main table within the Wikipedia article has been built

In [4]:
df2 = df.drop(columns=['Chinese', 'Area', 'Density'])

df2.drop([0], inplace=True)

print('Done')

Done


### Extra columns, unnecessary to the analysis I am performing, were removed from the data frame

In [5]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0


client_126b0639ffa14aa4bed65c2334e1e3a1 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='l7RLqK-YH8qSm6r85LOGpWL5L4ya1MkFqQrK-vwacPVz',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_126b0639ffa14aa4bed65c2334e1e3a1.get_object(Bucket='hongkongdataproject-donotdelete-pr-qwnqk60jrm2f4w',Key='Hong_Kong_District_Coordinates.csv')['Body']

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Hong_Kong = pd.read_csv(body)

Hong_Kong

,District,Region,Latitude,Longitude
0,Central and Western,Hong Kong Island,22.286667,114.155000
1,Eastern,Hong Kong Island,22.284167,114.224167
2,Southern,Hong Kong Island,22.247222,114.158889
3,Wan Chai,Kowloon,22.279722,114.171667
4,Sam Shui Po,Kowloon,22.330833,114.162222
5,Kowloon City,Kowloon,22.328333,114.191667
6,Kwun Tong,Kowloon,22.313333,114.225833
7,Wong Tai Sin,Kowloon,22.333611,114.196944
8,Yau Tsim Mong,Kowloon,22.321389,114.172500
9,Islands,New Territories,22.261111,113.946111


### I created a .csv file containing the latitude and longitude of each district in Hong Kong and uploaded it to the notebook

In [6]:
# The code was removed by Watson Studio for sharing.

Your credentails set


### It is at this point that I will upload my Foursquare credentials and begin the data visualization portion of the analysis

In [7]:
from folium import plugins

import seaborn as sns

%matplotlib inline 

import folium

print('done')

done


In [8]:
HK_map = folium.Map([22.3627778, 114.1291667], zoom_start=11)

print('Hong Kong Map Set')

Hong Kong Map Set


In [9]:
districts = folium.map.FeatureGroup()

for lat, lng, in zip(Hong_Kong.Latitude, Hong_Kong.Longitude):
    districts.add_child(
        folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='cloud',
                        color='red'),
    ).add_to(districts)
        )


HK_map.add_child(districts)

### Here I have attached icons to a folium map, indicating the locations of all individual districts within Hong Kong

In [10]:
address = '8 Connaught Place, Central, Hong Kong'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

22.2834451 114.1588156


### This is the address and coordinates to Hong Kong's stock exchange, the HKEX. This is the 6th largest stock exchange in the world, and third largest in East Asia/Asia. This will be the baseline for our comparison within Hong Kong, as it is the financial center of the city.

In [11]:
search_query = 'Hotel'
radius = 500
print(search_query + ' searched')

Hotel searched


In [12]:
HK_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

print('url set')

url set


In [13]:
HK_results = requests.get(HK_url).json()

print('done')

done


In [14]:
HK_venues = HK_results['response']['venues']

HK_venues_dataframe = json_normalize(HK_venues)

print('Hotels Set')

Hotels Set


In [15]:
HK_filtered_columns = ['name', 'categories'] + [col for col in HK_venues_dataframe.columns if col.startswith('location.')] + ['id']
HK_dataframe_filtered = HK_venues_dataframe.loc[:, HK_filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

HK_dataframe_filtered['categories'] = HK_dataframe_filtered.apply(get_category_type, axis=1)

HK_dataframe_filtered.columns = [column.split('.')[-1] for column in HK_dataframe_filtered.columns]

### The Foursquare analysis of all of the hotels surrounding HKEX has been completed

In [16]:
HKEX_Hotels = folium.Map(location=[latitude, longitude], zoom_start=16)

folium.Marker(
    [latitude, longitude],
    icon=folium.Icon(icon='cloud',
                        color='red'),
    popup='HKEX',
).add_to(HKEX_Hotels)

for lat, lng, label in zip(HK_dataframe_filtered.lat, HK_dataframe_filtered.lng, HK_dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(HKEX_Hotels)

HKEX_Hotels

In [17]:
HK_dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Four Seasons Hotel Hong Kong (香港四季酒店),Hotel,8 Finance St,HK,中環,香港,NaN,396,"[8 Finance St, 中西區, 香港]","[{'label': 'display', 'lat': 22.28655423619619...",22.286554,114.156929,Central,NaN,中西區,4b0588ccf964a52080da22e3
1,Mini Hotel,Hotel,32-38 Ice House St,HK,中環,香港,NaN,451,"[32-38 Ice House St, 中西區, 香港]","[{'label': 'display', 'lat': 22.28027566703209...",22.280276,114.156080,NaN,NaN,中西區,4f45843ce4b081614d77ab90
2,Lan Kwai Fong Hotel (蘭桂坊酒店),Hotel,3 Kau U Fong,HK,中環,香港,at 3 Aberdeen St,583,"[3 Kau U Fong (at 3 Aberdeen St), 中西區, 香港]","[{'label': 'display', 'lat': 22.28435824840652...",22.284358,114.153236,NaN,NaN,中西區,4b0588ccf964a52083da22e3
3,MTR Hong Kong Station (港鐵香港站),Metro Station,Harbour View St,HK,中環,香港,NaN,184,"[Harbour View St, 香港]","[{'label': 'display', 'lat': 22.28505407019187...",22.285054,114.158379,NaN,NaN,NaN,4b4e9d37f964a520a4f226e3
4,Mandarin Oriental Hong Kong (香港文華東方酒店),Hotel,"5 Connaught Road, Central",HK,中環,香港,NaN,185,"[5 Connaught Road, Central, 中西區, 香港]","[{'label': 'display', 'lat': 22.28187938916385...",22.281879,114.159443,Central,NaN,中西區,4c08db9f009a0f47efb1e6bf
5,Mini Hotel Hong Kong,Hotel Bar,"38 Ice House Street, Central / 8 Sun Wui Road,...",HK,香港,香港,NaN,78,"[38 Ice House Street, Central / 8 Sun Wui Road...","[{'label': 'display', 'lat': 22.2840731663032,...",22.284073,114.159161,NaN,000,NaN,5979acdcf427de3df29eb9d3
6,Four Seasons Hotel Grand Ballroom,Conference,NaN,HK,NaN,香港,NaN,402,[香港],"[{'label': 'display', 'lat': 22.286759, 'lng':...",22.286759,114.157256,NaN,NaN,NaN,573319b6498e5ae30c3c85e0
7,"The Hongkong & Shanghai Hotels, Ltd.",Office,"8/F St. George's Building, 2 Ice House St",HK,中環,香港,NaN,32,"[8/F St. George's Building, 2 Ice House St, 中西...","[{'label': 'display', 'lat': 22.28315866641874...",22.283159,114.158772,NaN,NaN,中西區,516b9051498e717afa73b73e
8,Hotel LKF Gym,Athletics & Sports,NaN,HK,NaN,香港,NaN,515,[香港],"[{'label': 'display', 'lat': 22.280397, 'lng':...",22.280397,114.155042,NaN,NaN,NaN,567e358f498e4fc4eeac4359
9,Madera Hollywood Hotel,Hotel,53-55 Hollywood Rd,HK,中環,香港,NaN,614,"[53-55 Hollywood Rd, 中西區, 香港]","[{'label': 'display', 'lat': 22.2828408837679,...",22.282841,114.152883,NaN,NaN,中西區,57061ae5498e2fee244ad1b9


### This would be one of the most important parts of the analysis. Above the map has a red cloud icon denoting the location of HKEX, with the blue circle markers indicating the hotels, and below is a dataframe listing the hotels in the immediate area.

## 2.2 Analysis of New York City
### Now is the beginning of the analysis of the comparative city, New York

In [18]:
address = '11 Wall St, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7070653 -74.0111743828446


### Beginning the analysis, we reset the location coordinates to the New York Stock Exchange

In [19]:
search_query = 'Hotel'
radius = 500
print(search_query + 's searched')

Hotels searched


In [20]:
NY_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

print('url set')

url set


In [21]:
NY_results = requests.get(NY_url).json()

print('done')

done


In [22]:
NY_venues = NY_results['response']['venues']

NY_venues_dataframe = json_normalize(NY_venues)

print('Hotels Set')

Hotels Set


In [23]:
NY_filtered_columns = ['name', 'categories'] + [col for col in NY_venues_dataframe.columns if col.startswith('location.')] + ['id']
NY_dataframe_filtered = NY_venues_dataframe.loc[:, NY_filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

NY_dataframe_filtered['categories'] = NY_dataframe_filtered.apply(get_category_type, axis=1)

NY_dataframe_filtered.columns = [column.split('.')[-1] for column in NY_dataframe_filtered.columns]

print('New York Analysis completed')

New York Analysis completed


### The same steps are performed for the New York analysis as the one for Hong Kong, with necessary adjustments completed

In [24]:
NYSE_Hotels = folium.Map(location=[latitude, longitude], zoom_start=16)

folium.Marker(
    [latitude, longitude],
    icon=folium.Icon(icon='cloud',
                        color='black'),
    popup='NYSE',
).add_to(NYSE_Hotels)

for lat, lng, label in zip(NY_dataframe_filtered.lat, NY_dataframe_filtered.lng, NY_dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(NYSE_Hotels)

NYSE_Hotels

In [25]:
NY_dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,DoubleTree by Hilton Hotel New York City - Fin...,Hotel,8 Stone St,US,New York,United States,NaN,361,"[8 Stone St, New York, NY 10004, United States]","[{'label': 'display', 'lat': 40.7039466, 'lng'...",40.703947,-74.012354,NaN,10004,NY,4cf021c094feb1f703420fba
1,"Club Quarters Hotel, Wall Street",Hotel,52 William St,US,New York,United States,NaN,205,"[52 William St, New York, NY 10005, United Sta...","[{'label': 'display', 'lat': 40.70711347511928...",40.707113,-74.008742,NaN,10005,NY,4a5e6955f964a52085be1fe3
2,Artezen Hotel,Hotel,24 John St,US,New York,United States,NaN,355,"[24 John St, New York, NY 10038, United States]","[{'label': 'display', 'lat': 40.70963898, 'lng...",40.709639,-74.008685,NaN,10038,NY,5a91fa58e679bc57ca94db72
3,The Beekman - A Thompson Hotel,Hotel,123 Nassau St,US,New York,United States,at Beekman St,592,"[123 Nassau St (at Beekman St), New York, NY 1...","[{'label': 'display', 'lat': 40.7111725, 'lng'...",40.711173,-74.006702,NaN,10038,NY,56d8c0f8498edb854f926e6a
4,World Center Hotel,Hotel,144 Washington St,US,New York,United States,NaN,374,"[144 Washington St, New York, NY 10006, United...","[{'label': 'display', 'lat': 40.7100640848828,...",40.710064,-74.013191,NaN,10006,NY,4b9180dcf964a5204dc133e3
5,Eurostar Hotel,Hotel Bar,NaN,US,New York,United States,NaN,478,"[New York, NY 10005, United States]","[{'label': 'display', 'lat': 40.70552230365685...",40.705522,-74.005886,NaN,10005,NY,4c449f16429a0f470500491e
6,"Club Quarters Hotel, World Trade Center",Hotel,140 Washington St,US,New York,United States,btwn Albany & Cedar,383,"[140 Washington St (btwn Albany & Cedar), New ...","[{'label': 'display', 'lat': 40.71001751931732...",40.710018,-74.013509,NaN,10006,NY,4b181dddf964a52033cd23e3
7,Q&A Residential Hotel,Hotel,70 Pine St,US,New York,United States,NaN,262,"[70 Pine St, New York, NY, United States]","[{'label': 'display', 'lat': 40.70655562577097...",40.706556,-74.008143,NaN,NaN,NY,566c66b838fa37fd4827c98e
8,Radisson Hotel New York Wall Street,Hotel,52 William St,US,New York,United States,NaN,191,"[52 William St, New York, NY 10005, United Sta...","[{'label': 'display', 'lat': 40.70676188163220...",40.706762,-74.008945,NaN,10005,NY,5d57978ea1d53e00085da67d
9,Beekman Hotel Fitness Center,Gym / Fitness Center,123 Nassau St,US,New York,United States,NaN,587,"[123 Nassau St, New York, NY 10038, United Sta...","[{'label': 'display', 'lat': 40.711129, 'lng':...",40.711129,-74.006741,NaN,10038,NY,59ce669c9411f21e8ca2192e


### The analysis of hotels near the New York Stock Exchange is visualized, with the hotels in the immediate area listed below. 

## 3.1 Results
### With a compilation of the top 30 hotels in each given area, we can see that Hong Kong has a higher number clustered close to the HKEX, with them also being clustered near the shore and some in town options, as oppose to the NYSE, which has them spread throughout the area.